# Reproduce FID and Directional loss
Please run the notebook below to reproduce the metrics reported in the blogpost.

In [1]:
import json
import os
import sys
from glob import glob
from pathlib import Path

import numpy as np
from PIL import Image

sys.path.append("../../src/lib/")
sys.path.append("../../src/lib/asyrp")

from lib_utils.metrics import DirectionalSimilarity, calculate_fid
from transformers import (CLIPImageProcessor, CLIPTextModelWithProjection,
                          CLIPTokenizer, CLIPVisionModelWithProjection)
from utils.text_dic import SRC_TRG_TXT_DIC

# see text in text_dic.py
GUID = "pixar"
DEVICE = "cuda:0"

(src_texts, target_texts) = SRC_TRG_TXT_DIC[GUID]

head_abl_path = Path("/home/parting/master_AI/DL2/DL2-2023-group-15/src/eval_runs/heads_ablation/")
layer_abl_path = Path("/home/parting/master_AI/DL2/DL2-2023-group-15/src/eval_runs/layertype_ablation/")
basepaths = [head_abl_path,layer_abl_path]
results = {}

clip_id = "openai/clip-vit-large-patch14"
tokenizer = CLIPTokenizer.from_pretrained(clip_id)
text_encoder = CLIPTextModelWithProjection.from_pretrained(clip_id).to(DEVICE)
image_processor = CLIPImageProcessor.from_pretrained(clip_id)
image_encoder = CLIPVisionModelWithProjection.from_pretrained(clip_id).to(DEVICE)

dir_similarity = DirectionalSimilarity(tokenizer, text_encoder, image_processor, image_encoder)

for basepath in basepaths:
    print(f'walking from: {basepath}')
    for path in os.walk(basepath, topdown=False):

        # find ablation paths
        if "40/edited" in str(path):
            print(f"parsing: {path[0]}")
            path = path[0] + "/"
            # path magic
            epoch = path.split("/40")[0][-1]
            ablation_type = path.split("eval_runs/")[-1]
            ablation_type = ablation_type.split("/40")[0]
            ablation_type, epoch = ablation_type[:-2], ablation_type[-1:]
            
            path_edited = path
            path_recon = path.replace("edited", "reconstructed")
            path_original = path.replace("edited", "original")

            list_original_path = glob(path_original + "/*.png")

            scores_or = []
            scores_oe = []
            scores_re = []

            for i in range(len(list_original_path)):
                img_original_path = str(Path(path_original) / f"test_{i}_19_ngen40_original.png")
                img_reconstructed_path = str(Path(path_recon) / f"test_{i}_19_ngen40_reconstructed.png")
                img_edited_path = str(Path(path_edited) / f"test_{i}_19_ngen40_edited.png")
                
                original_image = Image.open(img_original_path)
                reconstructed_image = Image.open(img_reconstructed_path)
                edited_image = Image.open(img_edited_path)
                
                similarity_score_or = dir_similarity(original_image, reconstructed_image, src_texts, src_texts)
                similarity_score_oe = dir_similarity(original_image, edited_image, src_texts, target_texts)
                similarity_score_re = dir_similarity(reconstructed_image, edited_image, src_texts, target_texts)

                scores_or.append(float(similarity_score_or.detach().cpu()))
                scores_oe.append(float(similarity_score_oe.detach().cpu()))
                scores_re.append(float(similarity_score_re.detach().cpu()))

            sdir_or = np.mean(scores_or)
            sdir_or_var = np.std(scores_or)
            sdir_oe = np.mean(scores_oe)
            sdir_oe_var = np.std(scores_oe)
            sdir_re = np.mean(scores_re)
            sdir_re_var = np.std(scores_re)
            print("=" * 50)
            print(f"Attribute {GUID} gives original-reconstructed CLIP directional similarity: {sdir_or} ({sdir_or_var})")
            print(f"Attribute {GUID} gives original-edited CLIP directional similarity: {sdir_oe}, ({sdir_oe_var})")
            print(f"Attribute {GUID} gives reconstructed-edited CLIP directional similarity: {sdir_re} ({sdir_re_var}) ")

            score_er = calculate_fid(path_edited, path_recon)
            score_ro = calculate_fid(path_original, path_recon)
            score_eo = calculate_fid(path_edited, path_original)
            print("calculating FID")

            print(f"ablation: '{ablation_type}' on epoch {epoch}, gives FID: {score_er} between edited and reconstructed")
            print(f"ablation: '{ablation_type}' on epoch {epoch}, gives FID: {score_eo} between edited and original")
            print(f"ablation: '{ablation_type}' on epoch {epoch}, gives FID: {score_ro} between reconstructed and original")
            print("-" * 50)

            run_results = {
                "sdir_or": float(sdir_or),
                "sdir_or_var": float(sdir_or_var),
                "sdir_oe": float(sdir_oe),
                "sdir_oe_var": float(sdir_oe_var),
                "sdir_re": float(sdir_re),
                "sdir_re_var": float(sdir_re_var),
                "fid_edited_reconstructed": score_er,
                "fid_edited_original": score_eo,
                "fid_reconstructed_original": score_ro,
                "epochs": epoch,
                "ablation_name": ablation_type,
                "attr": GUID
            }
            
            results[f"{ablation_type}_{epoch}_{GUID}"] = run_results

2023-05-27 13:47:33.913257: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-27 13:47:34.490844: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModelWithProjection: ['vision_model.encoder.layers.5.mlp.fc2.weight', 'vision_model.encoder.layers.22.self_attn.v_proj.bias', 'vision_model.encoder.layers.14.self_attn.out_proj.weight', 'vision_model.encoder.layers.3.self_attn.v_proj.bias', 'vision_model.encoder.layers.8.self_attn.out_proj.weight', 'vision_model.encoder.layers.22.self_attn.k_proj.bias', 'vision_model.encoder.layers.12.mlp.fc1.weight', 'vision_model.encoder.layers.4.self_attn.v_proj.w

walking from: /home/parting/master_AI/DL2/DL2-2023-group-15/src/eval_runs/heads_ablation
parsing: /home/parting/master_AI/DL2/DL2-2023-group-15/src/eval_runs/heads_ablation/h2/3/40/edited
Attribute pixar gives original-reconstructed CLIP directional similarity: 0.0 (0.0)
Attribute pixar gives original-edited CLIP directional similarity: 0.052924112514592704, (0.02865201260124684)
Attribute pixar gives reconstructed-edited CLIP directional similarity: 0.010375436714384704 (0.034696649130475564) 


parsing target images for fid: 100%|██████████| 100/100 [00:00<00:00, 571.69it/s]


calculating FID
ablation: 'heads_ablation/h2' on epoch 3, gives FID: 63.51802062988281 between edited and reconstructed
ablation: 'heads_ablation/h2' on epoch 3, gives FID: 83.07181549072266 between edited and original
ablation: 'heads_ablation/h2' on epoch 3, gives FID: 80.81753540039062 between reconstructed and original
--------------------------------------------------
parsing: /home/parting/master_AI/DL2/DL2-2023-group-15/src/eval_runs/heads_ablation/h2/4/40/edited
Attribute pixar gives original-reconstructed CLIP directional similarity: 0.0 (0.0)
Attribute pixar gives original-edited CLIP directional similarity: 0.05424981701653451, (0.029077282131697534)
Attribute pixar gives reconstructed-edited CLIP directional similarity: 0.011815618958789855 (0.03524624910270311) 


parsing target images for fid: 100%|██████████| 100/100 [00:00<00:00, 565.72it/s]


...

calculating FID
ablation: 'layertype_ablation/p_transformer_simple' on epoch 0, gives FID: 98.7154312133789 between edited and reconstructed
ablation: 'layertype_ablation/p_transformer_simple' on epoch 0, gives FID: 89.50467681884766 between edited and original
ablation: 'layertype_ablation/p_transformer_simple' on epoch 0, gives FID: 80.81753540039062 between reconstructed and original
--------------------------------------------------
parsing: /home/parting/master_AI/DL2/DL2-2023-group-15/src/eval_runs/layertype_ablation/p_transformer_simple/2/40/edited
Attribute pixar gives original-reconstructed CLIP directional similarity: 0.0 (0.0)
Attribute pixar gives original-edited CLIP directional similarity: -0.011198547976091504, (0.027105878506585822)
Attribute pixar gives reconstructed-edited CLIP directional similarity: -0.05455230691470206 (0.030932388993328737) 


parsing target images for fid: 100%|██████████| 100/100 [00:00<00:00, 601.15it/s]


calculating FID
ablation: 'layertype_ablation/p_transformer_simple' on epoch 2, gives FID: 108.02643585205078 between edited and reconstructed
ablation: 'layertype_ablation/p_transformer_simple' on epoch 2, gives FID: 112.72565460205078 between edited and original
ablation: 'layertype_ablation/p_transformer_simple' on epoch 2, gives FID: 80.81753540039062 between reconstructed and original
--------------------------------------------------
parsing: /home/parting/master_AI/DL2/DL2-2023-group-15/src/eval_runs/layertype_ablation/p_transformer_simple/1/40/edited
Attribute pixar gives original-reconstructed CLIP directional similarity: 0.0 (0.0)
Attribute pixar gives original-edited CLIP directional similarity: -0.004163689836859703, (0.023810866173065243)
Attribute pixar gives reconstructed-edited CLIP directional similarity: -0.04611961293965578 (0.027472759424438005) 


parsing target images for fid: 100%|██████████| 100/100 [00:00<00:00, 599.38it/s]


calculating FID
ablation: 'layertype_ablation/p_transformer_simple' on epoch 1, gives FID: 95.32453918457031 between edited and reconstructed
ablation: 'layertype_ablation/p_transformer_simple' on epoch 1, gives FID: 89.91558074951172 between edited and original
ablation: 'layertype_ablation/p_transformer_simple' on epoch 1, gives FID: 80.81753540039062 between reconstructed and original
--------------------------------------------------


In [2]:
# write results to json file
with open("metrics.json", "w") as f:
    f.write(json.dumps(results, indent=4))
    results